#Part 1 : Training a Name-Generating LSTM
First, we need to train an LSTM on a large dataset of names, so it can generate artificial names by predicting the nth character given (n-1) characters of a name.


## Imports and installs

In [0]:
#@title Importing `pandas`, `numpy`, `random` and `sys`
import pandas as pd
import numpy as np
import random
import sys

In [0]:
#@title Downloading dataset :  Baby Names from Social Security Card Applications - National Level Data 
!wget https://raw.githubusercontent.com/jcbain/celeb_baby_names/master/data/NationalNames.csv

--2018-12-08 12:10:16--  https://raw.githubusercontent.com/jcbain/celeb_baby_names/master/data/NationalNames.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 44350518 (42M) [text/plain]
Saving to: ‘NationalNames.csv’

NationalNames.csv   100%[===================>]  42.30M  39.0MB/s    in 1.1s    

2018-12-08 12:10:22 (39.0 MB/s) - ‘NationalNames.csv’ saved [44350518/44350518]



In [0]:
#@title Ignore warnings in output

import warnings
warnings.filterwarnings('ignore')

## Loading and pre-processing data. 

`SEQLEN` is the number of characters our LSTM uses to predict the next character</br>
`STEP` number of letters to skip between two samples

Hence, the name `PETER` is used to generate the following samples :

| X1  |  X2  | X3  | Y  |
|--:|---|---|---|
| -  | - | P  | **E**  |
| -  | P | E  | **T**  |
| P | E | T  | **E**  |
| E  | T | E  | **R**  |
| T  | E | R  | **-**  |

In [0]:
#@title Declaring `SEQLEN` and `STEP`
SEQLEN = 3
STEP = 1

- Loading names from `NationalNames.csv`
- Eliminating names shorter than 4 chars and having frequency less than 3
- Joining (seperating) names with `\n`

`Loaded 87659 names with 27 characters.`


In [0]:
#@title get_data
def get_data():
    df = pd.read_csv('NationalNames.csv')
    names = list(df[(df['Count'] > 3) & (df['Name'].str.len() > 4)]['Name'].unique())
    text = '\n' + '\n\n'.join(names).lower() + '\n'
    chars = sorted(list(set(text)))

    print ("Loaded",len(names),"names with",len(chars),"characters.")
    return text,chars

- Splitting text into `sequences` of 3 characters (X) and adding next character to `next_chars` (y)

```
No. of sequences: 764939
No. of chars: 764939
```

In [0]:
#@title get_seq
def get_seq(args):
    text = args[0]
    sequences = []
    next_chars = []
    for i in range(0, len(text) - SEQLEN, STEP):
        sequences.append(text[i: i + SEQLEN])
        next_chars.append(text[i + SEQLEN])
    print('No. of sequences:', len(sequences))
    print('No. of chars:', len(next_chars))
    return sequences,next_chars,args[1]


- One-Hot Encoding characters in `sequences` and `next_chars`

```
Shape of X (sequences): (764939, 3, 27)
Shape of y (next_chars): (764939, 27)
```

In [0]:
#@title get_vectors
def one_hot(word,char_indices):
    x_pred = np.zeros((1, SEQLEN, 27))

    for t, char in enumerate(word):
        x_pred[0, t, char_indices[char]] = 1.
    return x_pred
  
def get_vectors(args):
    sequences,next_chars,chars = args
    char_indices = dict((c, i) for i, c in enumerate(chars))
    indices_char = dict((i, c) for i, c in enumerate(chars))
    X = np.zeros((len(sequences), SEQLEN, len(chars)), dtype=np.bool)
    y = np.zeros((len(sequences), len(chars)), dtype=np.bool)
    for i, sentence in enumerate(sequences):
        X[i] = one_hot(sentence,char_indices)
        y[i, char_indices[next_chars[i]]] = 1
    print ("Shape of X (sequences):",X.shape)
    print ("Shape of y (next_chars):",y.shape)
    return X,y,char_indices, indices_char

## Creating the LSTM Model
- We're creating a simple LSTM model that takes in a sequence of size `SEQLEN`, each element of `len(chars)` numbers (1 or 0)
- The output of the LSTM goes into a Dense layer that predicts the next character with a softmaxed one-hot encoding

Note: While running on Google Colab, choose the TPU for training <br>(`Edit > Notebook settings > Hardware Accelerator > TPU`)

In [0]:
#@title Defining our model with Keras
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop

def get_model(num_chars):
    model = Sequential()
    model.add(LSTM(16, input_shape=(SEQLEN, num_chars)))
    model.add(Dense(num_chars))
    model.add(Activation('softmax'))

    optimizer = RMSprop(lr=0.01)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer)
    return model

Using TensorFlow backend.


In [0]:
#@title Loading the data and declaring our model
X,y,char_indices, indices_char = get_vectors(get_seq(get_data()))
model = get_model(len(char_indices.keys()))

Loaded 87659 names with 27 characters.
No. of sequences: 764939
No. of chars: 764939
Shape of X (sequences): (764939, 3, 27)
Shape of y (next_chars): (764939, 27)


In [0]:
#@title Training the model for 10 epochs

# On Google Colab, each epoch should take about 27 seconds
# For better results, you can run this cell again (and again)
model.fit(X, y,
          batch_size=128,
          epochs=40)

Epoch 1/40
  4608/764939 [..............................] - ETA: 3:15 - loss: 2.7143

KeyboardInterrupt: ignored

## Loading Pre-trained CharLSTM model

Alternatively you can also load a pretrained model

In [0]:
#@title Downloading model and weight files
!wget https://github.com/py-ranoid/WhatDoWeCallIt/raw/master/model_keras.h5 -nv
!wget https://github.com/py-ranoid/WhatDoWeCallIt/raw/master/model_keras.json -nv
  

2018-12-08 12:11:42 URL:https://raw.githubusercontent.com/py-ranoid/WhatDoWeCallIt/master/model_keras.h5 [25452/25452] -> "model_keras.h5" [1]
2018-12-08 12:11:44 URL:https://raw.githubusercontent.com/py-ranoid/WhatDoWeCallIt/master/model_keras.json [1582/1582] -> "model_keras.json" [1]


In [0]:
X,y,char_indices, indices_char = get_vectors(get_seq(get_data()))

#@title Load Pretained Model (optional, if model has been saved)
from keras.models import model_from_json

try:
  json_file = open('model_keras.json', 'r')
  loaded_model_json = json_file.read()
  json_file.close()
  model = model_from_json(loaded_model_json)
  model.load_weights("model_keras.h5")
  print("Loaded model from disk")  
  
except FileNotFoundError:
  print ("Models not found. Train them, maybe ?")

Loaded 87659 names with 27 characters.
No. of sequences: 764939
No. of chars: 764939
Shape of X (sequences): (764939, 3, 27)
Shape of y (next_chars): (764939, 27)
Loaded model from disk


## Sampling with our model
- Picking the element with the greatest probability will always return the same character for a given sequence
- I'd like to induce some variance by sampling from a probability array instead.

To explain this better, here's an excerpt  from Andrej Karpathy's blog aobut CharRNNs : 
> Temperature. We can also play with the temperature of the Softmax during sampling. Decreasing the **temperature from 1 to some lower number (e.g. 0.5) makes the RNN more confident, but also more conservative** in its samples. Conversely, **higher temperatures will give more diversity but at cost of more mistakes** (e.g. spelling mistakes, etc). In particular, setting temperature very near zero will give the most likely thing that Paul Graham might say:

> *“is that they were all the same thing that was a startup is that they were all the same thing that was a startup is that they were all the same thing that was a startup is that they were all the same”*







In [0]:
#@title Sampling and Generating functions
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def gen_name(seed):
    generated = seed
    for i in range(10):
        x_pred = np.zeros((1, SEQLEN, 27))
        for t, char in enumerate(seed):
            x_pred[0, t, char_indices[char]] = 1.
        preds = model.predict(x_pred, verbose=0)[0]
        next_char = indices_char[sample(preds,0.5)]
        if next_char == '\n':break
        generated += next_char
        seed = seed[1:] + next_char

    return generated

In [0]:
for i in ['dav','ram','seb']:
    print ('Seed: "'+i+'"\tNames :',[gen_name(i) for _ in range(5)]) 

Seed: "dav"	Names : ['davia', 'davia', 'davia', 'davetta', 'davia']
Seed: "ram"	Names : ['ram', 'ramia', 'ramiriann', 'ramir', 'ramari']
Seed: "seb"	Names : ['seb', 'seberta', 'seberta', 'seberniann', 'seboria']


# Part 2 : Use a CharLSTM model to join words

- `sample_preds` : Probabilities of 27 characters (A-Z + \n) to follow given sequence
- `ohmygauss`:Function to return decreasing gaussian sequence (right half of bell curve)

In [0]:
from scipy.stats import norm
def sample_preds(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    return preds


def ohmygauss(length, sigma=1.8):
    rv = norm(loc=0, scale=sigma)
    x = np.arange(length)
    return rv.pdf(x)


## Getting bridge scores
1. Iterate over all sequences of 3 (`SEQLEN`) characters in left word. (`MINLEFT -> n`)
  1. Iterate over all sequences of 3 (`COMPARE`) characters in right word. (`0 -> MINRIGHT`)
    1. Get probability that given character in right word will follow sequence from word
    2. Repeat for `COMPARE` sequences.</br>
    For example : to bridge **britain** and **exit** at `_br+exit`, <br>
     Score : `prob(e|"_br")*w1 + prob(x|"bre")*w2 + prob(i|"rex")*w3`
     3. Multiply Gaussian factors to score to prioritize words that are bridges towards the beggining of the right word

In [0]:
#@title Define `proc` to get bridge scores using LSTMs
MINLEFT = 3
MINRIGHT = 3
COMPARE = 3
LEFT_BIAS = [0.06, 0.05, 0.04]

def proc(left, right, verbose=False):
    best_matches = {}
    best_i = None
    best_i_score = -1
    for i in range(0, len(left) - MINLEFT + 1):
        # Searching all sequences of size COMPARE in the right word
        # to find best match
        best_j = None
        best_j_score = -1
        best_matches[i] = {}
        right_bound = len(right) - MINRIGHT + 1
        gaus_factors = ohmygauss(right_bound)
        for j in range(0, right_bound):
            right_chars = right[j:j + COMPARE]
            s = 0
            for x in range(COMPARE):
                
                # Character on right which is being sampled
                c_index = char_indices[right_chars[x]]
                if verbose:
                    print ("Sampling " + left[i + x:i + SEQLEN] +
                           right[j:j + x] + "-->" + right_chars[x])

                # Generating sequence and getting probability
                Xoh = one_hot(left[i + x:i + SEQLEN] + right[j:j + x],char_indices)
                preds = model.predict(Xoh, verbose=0)[0]
                pred_probs = sample_preds(preds, 0.7)

                # Getting corresponding character in left word
                left_char = np.zeros((1, len(char_indices)))
                try:
                    left_char[0, char_indices[left[i + SEQLEN + x]]] = 1
                except IndexError:
                    pass
                # Adding some bias to left_char and adding it to predicted probs
                biased_probs = LEFT_BIAS[x] * left_char + \
                    (1 - LEFT_BIAS[x]) * pred_probs

                # Adding probability of bridging at c_index to s
                s += biased_probs[0, c_index]

            # Prioritizing words that start with the first few letters of the right word
            s = s * gaus_factors[j]

            if verbose:
                print (i, j, s,)
            best_matches[i][j] = s
            if s > best_j_score:
                best_j = j
                best_j_score = s
#         best_matches[i] = {'index': best_j, 'score': best_j_score}
        if best_j_score > best_i_score and i < len(left) - MINLEFT:
            best_i_score = best_j_score
            best_i = i

    return best_matches, best_i

## Picking the best portmanteaus

- Maximize smoothness of the bridge (derived from `proc` using the LSTM  model)
- Minimize length of portmanteau
- Maximize fraction of each word in portmanteau

In [0]:
#@title Define `join` to pick best portmanteaus

SEQLEN = 3
MAXLEN = 10
PHONEME_WT = 4

def join(left, right, verbose=False,dict_result=False,n=3):
    left = '\n' + left
    right = right + '\n'
    matches, i = proc(left, right, verbose)
    probs = {}
    for i_temp in matches:
        for j_temp in matches[i_temp]:
            word = (left[:i_temp + SEQLEN] + right[j_temp:]).replace('\n', '').title()
            num_letters = len(word)
            if verbose :
                print (word, num_letters,(1 / float(num_letters)) * 0.5)
            probs[word] = probs.get(word,0)+round(matches[i_temp][j_temp],4) + (1 / float(num_letters) * PHONEME_WT)
            probs[word] *= (min((i_temp+1)/min(len(left),8),1.0) + min((len(right) - j_temp - 1)/min(len(right),8),1.0))
    if dict_result:
        return probs
    else:
        ser = pd.Series(probs).sort_values()[::-1][:n]
        ports = ser.index.tolist()
        port_vals = [i+'('+str(round(ser[i],3))+')' for i in ports]
        print (left,'+',right,' = ',port_vals)

In [0]:
pokemon_pairs = [('char','lizard'), ('venus', 'dinosaur'), ('blast', 'tortoise'), ('pikapika', 'chu')]        
word_pairs =  [('britain','exit'),('biology', 'electronic'), ('affluence', 'influenza'), ('brad', 'angelina'),
               ('brother', 'romance'), ('breakfast', 'lunch'), ('chill', 'relax'), ('emotion', 'icon'),('feminist', 'nazi')]

for p in word_pairs:
  join(p[0],p[1])


britain + exit
  =  ['Britainexit(0.71)', 'Brexit(0.705)', 'Briexit(0.69)']

biology + electronic
  =  ['Biolectronic(1.23)', 'Biolonic(0.821)', 'Bionic(0.677)']

affluence + influenza
  =  ['Affluenza(2.722)', 'Affluenfluenza(1.261)', 'Affluencenza(1.093)']

brad + angelina
  =  ['Brangelina(1.626)', 'Braangelina(0.637)', 'Bradangelina(0.635)']

brother + romance
  =  ['Brotheromance(1.493)', 'Bromance(0.963)', 'Brothermance(0.625)']

breakfast + lunch
  =  ['Breaunch(0.657)', 'Breakfasunch(0.59)', 'Breakfalunch(0.588)']

chill + relax
  =  ['Chillax(1.224)', 'Chilax(1.048)', 'Chillelax(0.699)']

emotion + icon
  =  ['Emoticon(1.331)', 'Emotion(0.69)', 'Emicon(0.667)']

feminist + nazi
  =  ['Feminazi(1.418)', 'Femazi(0.738)', 'Feministazi(0.678)']


# Result

In [0]:
#@title Demo Time!

left = 'chocolate' #@param {type:"string"}
right = 'alcoholic' #@param {type:"string"}
num_portmanteaus = 2 #@param {type:"slider", min:0, max:10, step:1}
join(left,right,n=num_portmanteaus)


chocolate + alcoholic
  =  ['Chocoholic(1.558)', 'Chocolalcoholic(1.351)']
